# Dynamic Convolutional Neural Network for sequential data

In [ ]:
import os
import sys
from pathlib import Path 

import numpy as np

import torch
from torch import nn

from torchsummary import summary



In [ ]:
#model 
KTOP = 2

def get_dmaxp(d, in_size, tot_L):
    m = (tot_L - in_size) * d // tot_L
    return max(KTOP, m)

def conv1dOShape(l_in, k, s=2, p=0):
    return (l_in - k + 2*p) // s + 1

class DCNN(nn.Module):

    def __init__(self, d, tot_l=2, wide=False):
        super(DCNN, self).__init__()
        self._d = d
        self._k = self._d + 3 if wide else 3 # input size s^{m x d} d number of features
        self._tot_l = tot_l
        
        self.model = nn.Sequential(
            nn.Conv1d(1, 5, self._k, stride=2),
            nn.LeakyReLU(),
            nn.MaxPool1d(get_dmaxp(d, conv1dOShape(self._d, self._k), self._tot_l)),
            nn.Conv1d(5, 3, 3, stride=2),
            nn.LeakyReLU(),
            nn.MaxPool1d(get_dmaxp(d, conv1dOShape(conv1dOShape(self._d, self._k), 3), self._tot_l)),
            nn.Flatten(0),
            nn.Linear(3, 2),
            nn.LeakyReLU(),
            nn.Softmax(dim=0)
            )
        
    def forward(self, x):
        return self.model(x)
    

In [ ]:
d = 24
x = torch.randn(1, d)
net = DCNN(d)
# summary(net, x.shape)

In [ ]:
conv1 = nn.Conv1d(1, 5, 3, stride=2)
maxp1 = nn.MaxPool1d(get_dmaxp(d, conv1dOShape(d, 3), 2))
conv2 = nn.Conv1d(5, 3, 3, stride=2)
maxp2 = nn.MaxPool1d(get_dmaxp(d, conv1dOShape(conv1dOShape(d, 3), 3), 2))
fl = nn.Flatten(0)
ln = nn.Linear(3, 2)
sm = nn.Softmax(dim=0)

x1 = conv1(x)
x2 = maxp1(x1)
x3 = conv2(x2)
x4 = maxp2(x3)
x5 = fl(x4)
x6 = ln(x5)
out = sm(x6)
out

In [ ]:
x.shape, x1.shape, x2.shape, x3.shape, x4.shape, x5.shape, out.shape